In [4]:
import pandas as pd
import json

# 🥗 1. Produtos em PT (já filtrados)
df_pt_food = pd.read_parquet("../../datasets/openfoods/produtos_openfoodfacts_pt_br.parquet")

df_insa_pt_food = pd.read_parquet("../../datasets/fsda/fsda_insa_dataset_final.parquet")

In [6]:
## Process df_insa_pt_food
## Keep only lines with LANGUAGE == 'pt'
df_insa_pt_food = df_insa_pt_food[df_insa_pt_food['LANGUAGE'] == 'PT']

# 2️⃣ Criar set para armazenar ingredientes únicos
ingredientes_set = set()

# 3️⃣ Separar ingredientes de cada linha e adicionar ao set
for row in df_insa_pt_food['FOOD_INGREDIENTS']:
    ingredientes = [i.strip().lower() for i in row.split(',') if i.strip()]
    ingredientes_set.update(ingredientes)

# 4️⃣ Converter para DataFrame final
df_ingredientes_unicos = pd.DataFrame(list(ingredientes_set), columns=['text'])

               text  label
0        clementina      1
1  açafrão-da-índia      1
2   feijão-manteiga      1
3              raia      1
4              coco      1
✅ Total de ingredientes únicos: 525


In [7]:
# Garantir que os nomes das colunas fiquem iguais
df_insa_pt_food_final = df_insa_pt_food[['FOOD_NAME']].rename(columns={'FOOD_NAME': 'text'})
df_pt_food_final = df_pt_food[['product_name']].rename(columns={'product_name': 'text'})
df_ingredientes_unicos_final = df_ingredientes_unicos[['text']]  # Já está certo

# Juntar os 3 datasets
df_pos_final = pd.concat([
    df_insa_pt_food_final,
    df_pt_food_final,
    df_ingredientes_unicos_final
], ignore_index=True)

# Remover duplicados caso exista overlap
df_pos_final = df_pos_final.drop_duplicates(subset="text").reset_index(drop=True)

# Adicionar a coluna isFood = 1
df_pos_final['isFood'] = 1

print(df_pos_final.head())
print(f"✅ Dataset final de positivos tem {len(df_pos_final)} linhas")


                   text  isFood
0          Abacate Hass       1
1  Abóbora cristalizada       1
2          Abóbora crua       1
3        Abrótea cozida       1
4          Abrótea crua       1
✅ Dataset final de positivos tem 31674 linhas


In [8]:
df_insa_pt_food_original = pd.read_parquet("../../datasets/fsda/fsda_insa_dataset_final.parquet")

# 1️⃣ Filtrar o dataset original para EN
df_insa_en_food = df_insa_pt_food_original[df_insa_pt_food_original['LANGUAGE'] == 'EN']

# 2️⃣ Balancear - pegar o mesmo número de linhas do PT
n_pt = len(df_pos_final)  # já filtrado anteriormente em PT

df_insa_en_food = df_insa_en_food.sample(n=n_pt, random_state=42)

# 3️⃣ Padronizar coluna para merge
df_insa_en_food_final = df_insa_en_food[['FOOD_NAME']].rename(columns={'FOOD_NAME': 'text'})
df_insa_en_food_final['isFood'] = 1

print(df_insa_en_food_final.head())
print(f"✅ Balanceado com {len(df_insa_en_food_final)} linhas em EN")

                                                     text  isFood
135346                                    YOGURT PRETZELS       1
76590               EL YUCATECO, CHILE HABANERO HOT SAUCE       1
18012   STARBUCKS CARAMEL MEDIUM ROAST LATIN AMERICAN ...       1
254659                 SNYDER'S OF HANOVER, MINI PRETZELS       1
80305        SPICY CHEESE STUFFED MEATBALLS, SPICY CHEESE       1
✅ Balanceado com 31674 linhas em EN


In [9]:
# Juntar os 2 datasets
df_positive_final = pd.concat([
    df_insa_en_food_final,
    df_pos_final
], ignore_index=True)
print(f"✅ Dataset final de positivos tem {len(df_positive_final)} linhas")

✅ Dataset final de positivos tem 63348 linhas


In [11]:
import json

n_pos = 63348 // 2  # Metade PT e metade EN

# 1️⃣ Carregar PT negativos
with open("../../datasets/classifier/pt.jsonl", "r", encoding="utf-8") as f:
    pt_neg = [json.loads(line) for line in f]
df_pt_neg = pd.DataFrame(pt_neg)
df_pt_neg = df_pt_neg.sample(n=n_pos, random_state=42)
df_pt_neg['isFood'] = 0

# 2️⃣ Carregar EN negativos
with open("../../datasets/classifier/en.jsonl", "r", encoding="utf-8") as f:
    en_neg = [json.loads(line) for line in f]
df_en_neg = pd.DataFrame(en_neg)
df_en_neg = df_en_neg.sample(n=n_pos, random_state=42)
df_en_neg['isFood'] = 0

# 3️⃣ Concatenar
df_neg_final = pd.concat([df_pt_neg, df_en_neg], ignore_index=True)

print(df_neg_final.head())
print(f"✅ Total de negativos balanceados: {len(df_neg_final)}")


                                                text  isFood
0  Depois do domínio do imperador Carlos Magno, s...       0
1  Quantos católicos sabem explicar o que é a Ima...       0
2  A prova do segundo teorema é obtida pela forma...       0
3  Descoberta da transformação Frederick Griffith...       0
4  Seres humanos e vários animais percebem sons c...       0
✅ Total de negativos balanceados: 63348


In [13]:
# 1️⃣ Unificar positivos + negativos
df_binario_final = pd.concat([df_positive_final, df_neg_final], ignore_index=True)

# 2️⃣ Shuffle geral
df_binario_final = df_binario_final.sample(frac=1, random_state=42).reset_index(drop=True)

# 3️⃣ Salvar em Parquet
df_binario_final.to_parquet("dataset_binario_food_detection.parquet", index=False)

print(f"✅ Dataset final salvo com {len(df_binario_final)} linhas (balanceado) em Parquet.")

✅ Dataset final salvo com 126696 linhas (balanceado) em Parquet.
